In [238]:
from arctic import Arctic, CHUNK_STORE
from arctic.date import DateRange
import pandas as pd
from ib_insync import IB, ContFuture, util, Future, Forex, Contract
from typing import Union
import pickle

In [73]:
util.startLoop()
ib = IB()
ib.connect(port=4002, clientId=6)

<IB connected to 127.0.0.1:4002 clientId=6>

In [80]:
nq = ContFuture('NQ', 'globex')

In [81]:
ib.qualifyContracts(nq)

[ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')]

In [82]:
nq

ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')

In [90]:
nq.dict()

{'secType': 'CONTFUT',
 'conId': 346577750,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20200320',
 'strike': 0.0,
 'right': '',
 'multiplier': '20',
 'exchange': 'GLOBEX',
 'primaryExchange': '',
 'currency': 'USD',
 'localSymbol': 'NQH0',
 'tradingClass': 'NQ',
 'includeExpired': False,
 'secIdType': '',
 'secId': '',
 'comboLegsDescrip': '',
 'comboLegs': None,
 'deltaNeutralContract': None}

In [91]:
nq.nonDefaults()

{'secType': 'CONTFUT',
 'conId': 346577750,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20200320',
 'multiplier': '20',
 'exchange': 'GLOBEX',
 'currency': 'USD',
 'localSymbol': 'NQH0',
 'tradingClass': 'NQ'}

In [200]:
ib.getCommissionBySymbol(nq)

AttributeError: 'IB' object has no attribute 'getCommissionBySymbol'

In [47]:
store = Arctic('localhost')

In [215]:
dir(store)

['DB_PREFIX',
 'METADATA_COLL',
 'METADATA_DOC_ID',
 '_Arctic__conn',
 '_MAX_CONNS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adminDB',
 '_allow_secondary',
 '_application_name',
 '_cache',
 '_conn',
 '_connect_timeout',
 '_given_instance',
 '_library_cache',
 '_list_libraries',
 '_list_libraries_cached',
 '_lock',
 '_pid',
 '_pymongo_kwargs',
 '_sanitize_lib_name',
 '_server_selection_timeout',
 '_socket_timeout',
 'check_quota',
 'delete_library',
 'get_library',
 'get_library_type',
 'get_quota',
 'initialize_library',
 'is_caching_enabled',
 'library_exists',
 'list_libraries',
 'mongo_host',
 'reload_cache

In [79]:
store.list_libraries()

['ib.history', 'history']

In [216]:
store.delete_library('ib.history')

In [217]:
store.list_libraries()

['history']

In [185]:
edek = Forex('EURUSD',)
ib.qualifyContracts(edek)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

In [186]:
bars = util.df(ib.reqHistoricalData(
        edek,
        endDateTime='',
        durationStr='5 D',
        barSizeSetting='1 hour',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1)).set_index('date')

In [187]:
bars

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-03-08 22:15:00,1.134375,1.135705,1.133600,1.134955,-1,-1.0,-1
2020-03-08 23:00:00,1.134955,1.139475,1.134005,1.138705,-1,-1.0,-1
2020-03-09 00:00:00,1.138705,1.139675,1.136680,1.138785,-1,-1.0,-1
2020-03-09 01:00:00,1.138785,1.138795,1.135905,1.135980,-1,-1.0,-1
2020-03-09 02:00:00,1.135980,1.149565,1.135610,1.142845,-1,-1.0,-1
...,...,...,...,...,...,...,...
2020-03-13 11:00:00,1.116520,1.117340,1.115305,1.115915,-1,-1.0,-1
2020-03-13 12:00:00,1.115915,1.117550,1.113965,1.115510,-1,-1.0,-1
2020-03-13 13:00:00,1.115510,1.118640,1.110920,1.111030,-1,-1.0,-1


In [50]:
store.initialize_library('history')
lib = store['history']

In [51]:
data = pd.read_pickle('/home/tomek/ib_tools/notebooks/data/minute_NQ_cont_non_active_included.pickle')
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2016-12-01 07:26:00,4818.00,4818.00,4818.00,4818.00,5,4818.000,4
2016-12-01 07:27:00,4818.00,4818.00,4818.00,4818.00,0,4818.000,0
2016-12-01 07:28:00,4818.00,4818.00,4818.00,4818.00,0,4818.000,0
2016-12-01 07:29:00,4818.00,4818.00,4818.00,4818.00,0,4818.000,0
2016-12-01 07:30:00,4818.00,4818.00,4818.00,4818.00,0,4818.000,0
...,...,...,...,...,...,...,...
2019-09-04 18:27:00,7695.50,7696.00,7694.75,7695.00,369,7695.475,178
2019-09-04 18:28:00,7695.00,7695.50,7694.75,7695.00,173,7695.025,72
2019-09-04 18:29:00,7695.00,7695.75,7694.50,7695.00,195,7695.125,87


In [52]:
data1 = data.iloc[100000:-50000]
data1

,open,high,low,close,volume,average,barCount
date,,,,,,,
2017-06-06 01:14:00,5888.75,5888.75,5888.75,5888.75,1,5888.75,1
2017-06-06 01:15:00,5888.75,5888.75,5888.75,5888.75,0,5888.75,0
2017-06-06 01:16:00,5888.75,5888.75,5888.75,5888.75,0,5888.75,0
2017-06-06 01:17:00,5888.75,5888.75,5888.75,5888.75,0,5888.75,0
2017-06-06 01:18:00,5888.75,5888.75,5888.75,5888.75,0,5888.75,0
...,...,...,...,...,...,...,...
2019-07-16 00:22:00,7982.75,7983.00,7982.50,7982.75,11,7982.85,5
2019-07-16 00:23:00,7982.50,7983.00,7982.50,7983.00,9,7982.80,6
2019-07-16 00:24:00,7983.00,7983.00,7983.00,7983.00,3,7983.00,2


In [119]:
meta = nq.nonDefaults()
meta.update({'repr': repr(nq)})

In [131]:
nq.dict()

{'secType': 'CONTFUT',
 'conId': 346577750,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20200320',
 'strike': 0.0,
 'right': '',
 'multiplier': '20',
 'exchange': 'GLOBEX',
 'primaryExchange': '',
 'currency': 'USD',
 'localSymbol': 'NQH0',
 'tradingClass': 'NQ',
 'includeExpired': False,
 'secIdType': '',
 'secId': '',
 'comboLegsDescrip': '',
 'comboLegs': None,
 'deltaNeutralContract': None}

In [239]:
type(nq)

ib_insync.contract.ContFuture

In [242]:
isinstance(nq, Forex)

False

In [134]:
nq_1 = Future(conId=nq.conId)
ib.qualifyContracts(nq_1)
nq_1

Future(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')

In [139]:
nq_1.secType

'FUT'

In [229]:
ib.reqContractDetails(nq)[0].nonDefaults()

{'contract': Contract(secType='CONTFUT', conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ'),
 'marketName': 'NQ',
 'minTick': 0.25,
 'orderTypes': 'ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,LTH,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,PEGBENCH,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF',
 'validExchanges': 'GLOBEX,QBALGO',
 'priceMagnifier': 1,
 'underConId': 11004958,
 'longName': 'E-mini NASDAQ 100 Futures',
 'contractMonth': '202003',
 'timeZoneId': 'CST (Central Standard Time)',
 'tradingHours': '20200312:1700-20200313:1515;20200313:1530-20200313:1600;20200314:CLOSED;20200315:1700-20200316:1515;20200316:1530-20200316:1600;20200316:1700-20200317:1515;20200317:1530-20200317:1600;20200317:1700-20200318:1515;20200318:1530-20200318:1600;20200318:

In [203]:
nq_pickle = pickle.dumps(nq)

In [204]:
nq_pickle

b'\x80\x03cib_insync.contract\nContFuture\nq\x00)\x81q\x01N}q\x02(X\x07\x00\x00\x00secTypeq\x03X\x07\x00\x00\x00CONTFUTq\x04X\x05\x00\x00\x00conIdq\x05JV[\xa8\x14X\x06\x00\x00\x00symbolq\x06X\x02\x00\x00\x00NQq\x07X\x1c\x00\x00\x00lastTradeDateOrContractMonthq\x08X\x08\x00\x00\x0020200320q\tX\x06\x00\x00\x00strikeq\nG\x00\x00\x00\x00\x00\x00\x00\x00X\x05\x00\x00\x00rightq\x0bX\x00\x00\x00\x00q\x0cX\n\x00\x00\x00multiplierq\rX\x02\x00\x00\x0020q\x0eX\x08\x00\x00\x00exchangeq\x0fX\x06\x00\x00\x00GLOBEXq\x10X\x0f\x00\x00\x00primaryExchangeq\x11h\x0cX\x08\x00\x00\x00currencyq\x12X\x03\x00\x00\x00USDq\x13X\x0b\x00\x00\x00localSymbolq\x14X\x04\x00\x00\x00NQH0q\x15X\x0c\x00\x00\x00tradingClassq\x16X\x02\x00\x00\x00NQq\x17X\x0e\x00\x00\x00includeExpiredq\x18\x89X\t\x00\x00\x00secIdTypeq\x19h\x0cX\x05\x00\x00\x00secIdq\x1ah\x0cX\x10\x00\x00\x00comboLegsDescripq\x1bh\x0cX\t\x00\x00\x00comboLegsq\x1cNX\x14\x00\x00\x00deltaNeutralContractq\x1dNu\x86q\x1eb.'

In [136]:
def symbol_maker(obj: Union[ContFuture, Future]) -> str:
    return f'{obj.localSymbol}_{obj.secType}'

In [192]:
def writer(obj: Union[ContFuture, Future], data: pd.DataFrame) -> str:
    return lib.write(symbol_maker(obj), data, metadata={**obj.nonDefaults(), **{'repr': repr(obj), 'secType': obj.secType}})

In [206]:
lib.write('test', data, metadata={'object': nq_pickle})

VersionedItem(symbol=test,library=arctic.history,data=<class 'NoneType'>,version=2,metadata={'object': b'\x80\x03cib_insync.contract\nContFuture\nq\x00)\x81q\x01N}q\x02(X\x07\x00\x00\x00secTypeq\x03X\x07\x00\x00\x00CONTFUTq\x04X\x05\x00\x00\x00conIdq\x05JV[\xa8\x14X\x06\x00\x00\x00symbolq\x06X\x02\x00\x00\x00NQq\x07X\x1c\x00\x00\x00lastTradeDateOrContractMonthq\x08X\x08\x00\x00\x0020200320q\tX\x06\x00\x00\x00strikeq\nG\x00\x00\x00\x00\x00\x00\x00\x00X\x05\x00\x00\x00rightq\x0bX\x00\x00\x00\x00q\x0cX\n\x00\x00\x00multiplierq\rX\x02\x00\x00\x0020q\x0eX\x08\x00\x00\x00exchangeq\x0fX\x06\x00\x00\x00GLOBEXq\x10X\x0f\x00\x00\x00primaryExchangeq\x11h\x0cX\x08\x00\x00\x00currencyq\x12X\x03\x00\x00\x00USDq\x13X\x0b\x00\x00\x00localSymbolq\x14X\x04\x00\x00\x00NQH0q\x15X\x0c\x00\x00\x00tradingClassq\x16X\x02\x00\x00\x00NQq\x17X\x0e\x00\x00\x00includeExpiredq\x18\x89X\t\x00\x00\x00secIdTypeq\x19h\x0cX\x05\x00\x00\x00secIdq\x1ah\x0cX\x10\x00\x00\x00comboLegsDescripq\x1bh\x0cX\t\x00\x00\x00comboLegs

In [207]:
t = lib.read('test')

In [214]:
pickle.loads(t.metadata['object'])

ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')

In [243]:
type(writer(nq_1, data1))

arctic.store.versioned_item.VersionedItem

ERROR:ib_insync.client:Peer closed connection
ERROR:ib_insync.client:Peer closed connection


In [194]:
writer(edek, bars)

VersionedItem(symbol=EUR.USD_CASH,library=arctic.history,data=<class 'NoneType'>,version=1,metadata={'secType': 'CASH', 'conId': 12087792, 'symbol': 'EUR', 'exchange': 'IDEALPRO', 'currency': 'USD', 'localSymbol': 'EUR.USD', 'tradingClass': 'EUR.USD', 'repr': "Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')"},host=localhost)

In [195]:
lib.list_symbols()

['EUR.USD_CASH', 'NQ', 'NQH0_FUT']

In [196]:
writer(nq, data)

VersionedItem(symbol=NQH0_CONTFUT,library=arctic.history,data=<class 'NoneType'>,version=1,metadata={'secType': 'CONTFUT', 'conId': 346577750, 'symbol': 'NQ', 'lastTradeDateOrContractMonth': '20200320', 'multiplier': '20', 'exchange': 'GLOBEX', 'currency': 'USD', 'localSymbol': 'NQH0', 'tradingClass': 'NQ', 'repr': "ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')"},host=localhost)

In [197]:
lib.list_symbols()

['EUR.USD_CASH', 'NQ', 'NQH0_CONTFUT', 'NQH0_FUT']

In [199]:
e = lib.read('EUR.USD_CASH')
e.metadata

{'secType': 'CASH',
 'conId': 12087792,
 'symbol': 'EUR',
 'exchange': 'IDEALPRO',
 'currency': 'USD',
 'localSymbol': 'EUR.USD',
 'tradingClass': 'EUR.USD',
 'repr': "Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')"}

In [235]:
e.data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-03-08 22:15:00,1.134375,1.135705,1.133600,1.134955,-1,-1.0,-1
2020-03-08 23:00:00,1.134955,1.139475,1.134005,1.138705,-1,-1.0,-1
2020-03-09 00:00:00,1.138705,1.139675,1.136680,1.138785,-1,-1.0,-1
2020-03-09 01:00:00,1.138785,1.138795,1.135905,1.135980,-1,-1.0,-1
2020-03-09 02:00:00,1.135980,1.149565,1.135610,1.142845,-1,-1.0,-1
...,...,...,...,...,...,...,...
2020-03-13 11:00:00,1.116520,1.117340,1.115305,1.115915,-1,-1.0,-1
2020-03-13 12:00:00,1.115915,1.117550,1.113965,1.115510,-1,-1.0,-1
2020-03-13 13:00:00,1.115510,1.118640,1.110920,1.111030,-1,-1.0,-1


In [236]:
lib.read('EUR.USD_CASH', date_range=DateRange('20200301','20200310')).data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2020-03-08 22:15:00,1.134375,1.135705,1.133600,1.134955,-1,-1.0,-1
2020-03-08 23:00:00,1.134955,1.139475,1.134005,1.138705,-1,-1.0,-1
2020-03-09 00:00:00,1.138705,1.139675,1.136680,1.138785,-1,-1.0,-1
2020-03-09 01:00:00,1.138785,1.138795,1.135905,1.135980,-1,-1.0,-1
2020-03-09 02:00:00,1.135980,1.149565,1.135610,1.142845,-1,-1.0,-1
2020-03-09 03:00:00,1.142845,1.146125,1.138660,1.139810,-1,-1.0,-1
2020-03-09 04:00:00,1.139810,1.142740,1.137140,1.141655,-1,-1.0,-1
2020-03-09 05:00:00,1.141655,1.142360,1.139755,1.140175,-1,-1.0,-1
2020-03-09 06:00:00,1.140175,1.142545,1.138110,1.138195,-1,-1.0,-1


In [147]:
item = lib.read('NQ')

In [151]:
item.version

12

In [152]:
item_1 = lib.read(symbol_maker(nq_1))

In [153]:
item_1

VersionedItem(symbol=NQH0_FUT,library=arctic.history,data=<class 'pandas.core.frame.DataFrame'>,version=1,metadata={'secType': 'FUT', 'conId': 346577750, 'symbol': 'NQ', 'lastTradeDateOrContractMonth': '20200320', 'multiplier': '20', 'exchange': 'GLOBEX', 'currency': 'USD', 'localSymbol': 'NQH0', 'tradingClass': 'NQ', 'repr': "Future(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')"},host=localhost)

In [154]:
item_1.version

1

In [148]:
item.metadata

{'secType': 'CONTFUT',
 'conId': 346577750,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20200320',
 'multiplier': '20',
 'exchange': 'GLOBEX',
 'currency': 'USD',
 'localSymbol': 'NQH0',
 'tradingClass': 'NQ',
 'repr': "ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')"}

In [149]:
lib.read_metadata('NQ').metadata

{'secType': 'CONTFUT',
 'conId': 346577750,
 'symbol': 'NQ',
 'lastTradeDateOrContractMonth': '20200320',
 'multiplier': '20',
 'exchange': 'GLOBEX',
 'currency': 'USD',
 'localSymbol': 'NQH0',
 'tradingClass': 'NQ',
 'repr': "ContFuture(conId=346577750, symbol='NQ', lastTradeDateOrContractMonth='20200320', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH0', tradingClass='NQ')"}

In [57]:
lib.list_symbols()

['NQ']

In [58]:
lib.append('NQ', data.iloc[-50000:])

VersionedItem(symbol=NQ,library=arctic.history,data=<class 'NoneType'>,version=2,metadata={'source': 'history', 'type': 'continuous'},host=localhost)

In [59]:
lib.read('NQ')

VersionedItem(symbol=NQ,library=arctic.history,data=<class 'pandas.core.frame.DataFrame'>,version=2,metadata={'source': 'history', 'type': 'continuous'},host=localhost)

In [61]:
lib.append('NQ', data.iloc[:100000])

VersionedItem(symbol=NQ,library=arctic.history,data=<class 'NoneType'>,version=3,metadata={'source': 'history', 'type': 'continuous'},host=localhost)

In [62]:
lib.read('NQ')

VersionedItem(symbol=NQ,library=arctic.history,data=<class 'pandas.core.frame.DataFrame'>,version=3,metadata={'source': 'history', 'type': 'continuous'},host=localhost)

In [26]:
nq_hourly = pd.read_pickle('/home/tomek/ib_tools/notebooks/data/hourly_NQ0919.pickle')

In [27]:
nq_hourly.head()

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-07-16 15:30:00,7509.25,7509.25,7509.25,7509.25,0,7509.25,0
2018-07-16 16:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.25,0
2018-07-16 17:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.25,0
2018-07-16 18:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.25,0
2018-07-16 19:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.25,0


In [28]:
%%timeit
lib.write('NQ_hour', nq_hourly[50:])

220 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
lib.read('NQ_hour')

355 ms ± 3.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
lib.write('NQ_hour_2', nq_hourly[50:], chunk_size='M')

17.5 ms ± 90.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%%timeit
lib.read('NQ_hour_2')

26.9 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
lib.append('NQ_hour_2', nq_hourly[:50])

In [33]:
lib.append('NQ_hour', nq_hourly[:50])

In [34]:
lib.read('NQ_hour')

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-07-16 15:30:00,7509.25,7509.25,7509.25,7509.25,0,7509.250,0
2018-07-16 16:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.250,0
2018-07-16 17:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.250,0
2018-07-16 18:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.250,0
2018-07-16 19:00:00,7509.25,7509.25,7509.25,7509.25,0,7509.250,0
...,...,...,...,...,...,...,...
2019-08-02 20:00:00,7661.25,7695.00,7652.75,7681.50,56300,7679.375,31642
2019-08-02 21:00:00,7681.75,7725.25,7665.75,7701.50,93099,7696.900,48215
2019-08-02 22:00:00,7701.50,7709.00,7691.25,7702.00,9766,7700.125,4636


In [237]:
lib.get_info('NQ')

{'size': 47222592,
 'segment_count': 23,
 'dtype': [('date', '<M8[ns]'),
  ('open', '<f8'),
  ('high', '<f8'),
  ('low', '<f8'),
  ('close', '<f8'),
  ('volume', '<i8'),
  ('average', '<f8'),
  ('barCount', '<i8')],
 'type': 'pandasdf',
 'handler': 'PandasDataFrameStore',
 'rows': 737853,
 'col_names': {'index': ['date'],
  'columns': ['open',
   'high',
   'low',
   'close',
   'volume',
   'average',
   'barCount']}}

In [64]:
lib.list_symbols()

['NQ']